<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with a custom metrics provider

This notebook demonstrates how to configure the custom monitor and custom metrics deployment by using IBM Watson OpenScale. This notebook should be run in a Watson Studio project, using **IBM Runtime 22.1 on Python 3.9 XS** runtime environment. **If you are viewing this in Watson Studio and do not see the required runtime env in the upper right corner of your screen, please update the runtime now.**. It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning

  
## Contents

This notebook contains the following parts:

  1. [Set up your environment.](#setup)
  1. [Create the custom metrics provider - python function.](#provider)
  1. [Register the custom metrics provider and create a deployment.](#deployment)
  1. [Configure Watson OpenScale](#config)
  1. [Create the integrated system for the custom metrics provider.](#custom)
  1. [Set up the custom monitor definition and instance.](#instance)


## 1. Set up your environment. <a name="setup"></a>

Before you use the sample code in this notebook, you must perform the following setup tasks:

### Install the  `ibm-watson-machine-learning` and `ibm-watson-openscale` packages.

In [ ]:
!pip install --upgrade ibm-watson-machine-learning | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

### Action: restart the kernel!

### Credentials for IBM Cloud Pak for Data
To authenticate, in the following code boxes, replace the sample data with your own credentials. Get the information from your system administrator or through the Cloud Pak for Data dashboard.


### Obtaining your Watson OpenScale credentials

You can retrieve the URL by running the following command: `oc get route -n namespace1 --no-headers | awk '{print $2}'` Replace the `namespace1` variable with your namespace.

You should have been assigned a username and password when you were added to the Cloud Pak for Data system. You might need to ask either your database administrator or your system administrator for some of the information.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
############################################################################################
# Paste your Watson OpenScale credentials into the following section and then run this cell.
############################################################################################
CLOUD_API_KEY = "<Your Cloud IAM API Key>"
WOS_CREDENTIALS = {
    "url": "https://api.aiopenscale.cloud.ibm.com",
    "apikey": CLOUD_API_KEY
}


In [ ]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [ ]:
#update your WML instance url
WML_CREDENTIALS = {
        "url": "https://us-south.ml.cloud.ibm.com",
        "apikey": CLOUD_API_KEY
}


### Enter your Watson OpenScale GUID.

For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.


In [ ]:
#Update your Watson OpenScale datamart id.
WOS_GUID="<IBM Watson OpenScale DataMart Id>"

### Define the Watson OpenScale subscription to which the custom metrics have to be sent.

Create a subscription from the Watson Openscale UI or SDK to configure custom metrics. You can configure custom metrics for the subscriptions that have predefined monitors, such as fairness, quality, or drift or without predefined monitors.

In [ ]:
####################################################################
# Paste your Subscription in the following field and then run this cell.
####################################################################

subscription_id = "<Subscription Id>"

### Python function details

In [ ]:
PYTHON_FUNCTION_NAME = 'Custom Metrics Provider Batch Function'
DEPLOYMENT_NAME = 'Custom Metrics Provider Batch Deployment'

### OpenScale Custom Metrics Provider name

In [ ]:
CUSTOM_METRICS_PROVIDER_NAME = "Custom Metrics Provider"

### OpenSale Custom Monitor name

In [ ]:
###################################################################
# UPDATE your custom monitor name in the following field and then run this cell.
####################################################################
CUSTOM_MONITOR_NAME = 'Sample Model Performance'

## 2. Create the custom metrics provider - Python function. <a name="provider"></a>

The Python function receives the required variables, such as the `datamart_id`, `monitor_instance_id`, `monitor_id`, `monitor_instance_parameters` and `subscription_id` from the Watson OpenScale service when it is invoked by the custom monitor. 

In the Python function, add your own logic to compute the custom metrics in the `get_metrics` method, publish the metrics to the Watson Openscale service and update the status of the run to the `finished` state in the custom monitor instance.

Update the `WOS_CREDENTIALS` in the Python function. 

In [ ]:
#wml_python_function
parms = {
        "iam_url": IAM_URL,
        "url": WOS_CREDENTIALS["url"],
        "apikey": WOS_CREDENTIALS["apikey"]
    }
def custom_metrics_provider(parms = parms):
    
    import json
    import requests
    import base64
    from requests.auth import HTTPBasicAuth
    import time
    import uuid    
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"
    
    
    # Get the access token
    def get_access_token():
        url = parms['iam_url']
        headers={}
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        headers["Accept"] = "application/json"
        data = {
             "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
             "apikey": parms['apikey']
        }
        response = requests.post(url, data=data, headers=headers)
        json_data = response.json()
        access_token = json_data['access_token']
        return access_token
    
    def get_feedback_dataset_id(access_token, data_mart_id, subscription_id):
        headers["Authorization"] = "Bearer {}".format(access_token)
        DATASETS_URL =  parms["url"] + "/openscale/{0}/v2/data_sets?target.target_id={1}&target.target_type=subscription&type=feedback".format(data_mart_id, subscription_id)
        response = requests.get(DATASETS_URL, headers=headers, verify=False)
        json_data = response.json()
        feedback_dataset_id = None
        if "data_sets" in json_data and len(json_data["data_sets"]) > 0:
            feedback_dataset_id = json_data["data_sets"][0]["metadata"]["id"]
        
        return feedback_dataset_id
    
    def get_feedback_data(access_token, data_mart_id, feedback_dataset_id):
        json_data = None
        if feedback_dataset_id is not None:
            headers["Authorization"] = "Bearer {}".format(access_token)
            DATASETS_STORE_RECORDS_URL = parms["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&format=list".format(data_mart_id, feedback_dataset_id, 100)
            response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
            json_data = response.json()
            return json_data
    
    #Update the run status to Finished in the custom monitor instance
    def update_monitor_instance(base_url, access_token, custom_monitor_instance_id, payload):
        monitor_instance_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '?update_metadata_only=true'
        
        patch_payload  = [
            {
                "op": "replace",
                "path": "/parameters",
                "value": payload
            }
        ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        response = requests.patch(monitor_instance_url, headers=headers, json = patch_payload, verify=False)
        monitor_response = response.json()
        return response.status_code, monitor_response
    
    #Add your code to compute the custom metrics. 
    def get_metrics(access_token, data_mart_id, subscription_id):
        #Add the logic here to compute the metrics. Use the below metric names while creating the custom monitor definition
        feedback_dataset_id = get_feedback_dataset_id(access_token, data_mart_id, subscription_id)
        json_data = get_feedback_data(access_token, data_mart_id, feedback_dataset_id)
        gender_less40_fav_prediction_ratio = 0
        if json_data is not None:
            fields = json_data['records'][0]['fields']
            values = json_data['records'][0]['values']
            import pandas as pd
            feedback_data = pd.DataFrame(values, columns = fields)
            female_less40_fav_prediction = len(feedback_data.query('Sex == \'female\' & Age <= 40 & Risk == \'No Risk\''))
            male_less40_fav_prediction = len(feedback_data.query('Sex == \'male\' & Age <= 40 & Risk == \'No Risk\''))
            gender_less40_fav_prediction_ratio = female_less40_fav_prediction / male_less40_fav_prediction

        metrics = {"specificity": 1.2, "sensitivity": 0.85, "gender_less40_fav_prediction_ratio": gender_less40_fav_prediction_ratio, "region": "us-south"}
        
        return metrics
        
        
    # Publishes the Custom Metrics to OpenScale
    def publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, timestamp):
        # Generate an monitoring run id, where the publishing happens against this run id
        custom_metrics = get_metrics(access_token, data_mart_id, subscription_id)
        measurements_payload = [
                  {
                    "timestamp": timestamp,
                    "run_id": custom_monitoring_run_id,
                    "metrics": [custom_metrics]
                  }
                ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        measurements_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/measurements'
        response = requests.post(measurements_url, headers=headers, json = measurements_payload, verify=False)
        published_measurement = response.json()
     
        return response.status_code, published_measurement
        
    
    def publish( input_data ):
        import datetime
        timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        
        payload_array = input_data.get("input_data")[0].get("values")
        payload = payload_array[0]
        data_mart_id = payload['data_mart_id']
        subscription_id = payload['subscription_id']
        custom_monitor_id = payload['custom_monitor_id']
        custom_monitor_instance_id = payload['custom_monitor_instance_id']
        custom_monitor_instance_params  = payload['custom_monitor_instance_params']

        base_url = parms['url'] + '/openscale' + '/' + data_mart_id
        access_token = get_access_token()
        
        published_measurements = []
        error_msgs = []
        
        custom_monitoring_run_id = custom_monitor_instance_params["run_details"]["run_id"]
        try:
            status_code, published_measurement = publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, timestamp)
            if int(status_code) in [200, 201, 202]:
                custom_monitor_instance_params["run_details"]["run_status"] = "finished"
                published_measurements.append(published_measurement)
            else:
                custom_monitor_instance_params["run_details"]["run_status"] = "error"
                custom_monitor_instance_params["run_details"]["run_error_msg"] = published_measurement
                error_msgs.append(published_measurement)
                    
            custom_monitor_instance_params["last_run_time"] = timestamp
            status_code, response = update_monitor_instance(base_url, access_token, custom_monitor_instance_id, custom_monitor_instance_params)
            if not int(status_code) in [200, 201, 202]:
                error_msgs.append(response)
                
        except Exception as ex:
            error_msgs.append(str(ex))
        if len(error_msgs) == 0:
            response_payload = {
                "predictions" : [{ 
                    "values" : published_measurements
                }]

            }
        else:
            response_payload = {
                "predictions":[],
                "errors": error_msgs
            }
        
        return response_payload
        
    return publish
    

## 3. Register the custom metrics provider and create a deployment. <a name="deployment"></a>

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.spaces.list(limit=10)

In [ ]:
space_id = "<Update your space id>"
wml_client.set.default_space(space_id)

### Remove existing function and deployment.

In [ ]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME or deployment["metadata"]["name"] == DEPLOYMENT_NAME + '_WRAPPER':
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)

wml_client.repository.list_functions()

### Get the software spec id


In [ ]:
software_spec_id =  wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
print(software_spec_id)

### Create the batch python function meta properties



In [ ]:
function_meta_props = {
     wml_client.repository.FunctionMetaNames.NAME: PYTHON_FUNCTION_NAME,
     wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_id
     }


### Store the Python function

In [ ]:
function_artifact = wml_client.repository.store_function(meta_props=function_meta_props, function=custom_metrics_provider)
function_uid = wml_client.repository.get_function_id(function_artifact)
print("Function UID = " + function_uid)

In [ ]:
function_details = wml_client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

### Deploy the Python function.


In [ ]:
hardware_spec_id = wml_client.hardware_specifications.get_id_by_name('M')
hardware_spec_id

### Create deployment metadata for the batch python function.

In [ ]:
deploy_meta = {
 wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
 wml_client.deployments.ConfigurationMetaNames.BATCH: {},
 wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "name": "S", "num_nodes": 1}
}

### Create the batch python function deployment.

In [ ]:
deployment_details = wml_client.deployments.create(function_uid, meta_props=deploy_meta)

In [ ]:
deployment_uid = wml_client.deployments.get_uid(deployment_details)
deployment_uid

### Get the batch deployment job URL.

In [ ]:
created_at = deployment_details['metadata']['created_at']
find_string_pos = created_at.find("T")
if find_string_pos != -1:
    current_date = created_at[0:find_string_pos]
scoring_url = WML_CREDENTIALS['url'] + '/ml/v4/deployment_jobs?version='+current_date
print(scoring_url)

## 4. Configure OpenScale. <a name="config"></a>

Import the required libraries and set up the Watson OpenScale Python client.

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMetricRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MetricThreshold
from ibm_watson_openscale.supporting_classes.enums import MetricThresholdTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorTagRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Target
from ibm_watson_openscale.supporting_classes.enums import TargetTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import IntegratedSystems

from datetime import datetime, timezone, timedelta
import uuid

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator)
wos_client.version


## 5. Create the integrated system for the custom metrics provider. <a name="custom"></a>


Update the custom metrics deployment URL, which is created during the Python function creation in the integrated system. Watson OpenScale invokes the deployment URL at runtime to compute the custom metrics. 

You must define the authentication type based on the communication with custom metrics deployment. Watson OpenScale supports 2 types of authentication: basic and bearer. If custom metrics deployment accepts the `basic` authentication type, then provide `auth_type=basic` otherwise use `auth_type=bearer`.

In [ ]:
auth_type = "bearer" #Supported values are basic and bearer

if auth_type == "basic":
    CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
        "auth_type":"basic",
        "username":  "*****",# update the username here 
        "password": "*****"# Update the password here
   }
    
if auth_type == "bearer":
    CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
        "auth_type":"bearer",
        "token_info": {
           "url":  IAM_URL,
           "headers": { "Content-type": "application/x-www-form-urlencoded" }, # update the headers here 
           "payload": "grant_type=urn:ibm:params:oauth:grant-type:apikey&response_type=cloud_iam&apikey="+CLOUD_API_KEY, # update the payload here 
           "method": "POST" 
        }        
    }
    
  #if custom metrics deployment is on other cpd cluster or some other cloud then please uncomment and update 
  #the below "TOKEN_INFO" properties to generate the token to communicate to the custom metrics deployment url
  #Here are the sample values given in the token_info
    #TOKEN_INFO = {
    #    "url":  "https://iam.ng.bluemix.net/oidc/token", # update the token generation here 
    #    "headers": { "Content-type": "application/x-www-form-urlencoded" }, # update the headers here 
    #    "payload": "grant_type=urn:ibm:params:oauth:grant-type:apikey&response_type=cloud_iam&apikey=<api_key>", # update the payload here 
    #    "method": "POST" # # update the http method here 
    #}
    #CUSTOM_METRICS_PROVIDER_CREDENTIALS["token_info"] = TOKEN_INFO


### Remove existing integrated system 

In [ ]:
# Delete existing custom metrics provider integrated systems if present
integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems
for system in integrated_systems:
    if system.entity.type == 'custom_metrics_provider' and system.entity.name == CUSTOM_METRICS_PROVIDER_NAME:
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

In [ ]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=CUSTOM_METRICS_PROVIDER_NAME,
    description=CUSTOM_METRICS_PROVIDER_NAME,
    type="custom_metrics_provider",
    credentials= CUSTOM_METRICS_PROVIDER_CREDENTIALS,
    connection={
        "display_name": CUSTOM_METRICS_PROVIDER_NAME,
        "endpoint": scoring_url
    }
).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(custom_metrics_integrated_system)

## 6. Set up the custom monitor definition and instance. <a name="instance"></a>


### Check for the existence of the custom monitor definition.


In [ ]:
def get_custom_monitor_definition():
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    for definition in monitor_definitions:
        if CUSTOM_MONITOR_NAME == definition.entity.name:
            return definition
    return None   

### Create the  custom monitor definition.

Update the custom metric names, threshold types (`LOWER_LIMIT`, `UPPER_LIMIT`) and default values as required. You can define the threshold type as lower limit, upper limit, or both.

In [ ]:
###################################################################
# Update your custom monitor metrics names in the following field. Use the same metric names for creating the 
# monitor definition and publishing the metrics to Openscale in your python function
####################################################################
CUSTOM_MONITOR_METRICS_NAMES = ['sensitivity','specificity', 'gender_less40_fav_prediction_ratio']
#Update the tag values if you want to fetch the metrics by tags
TAGS= ['region']
TAG_DESCRIPTION =['customer geographical region'] 

In [ ]:
#Update the Threshold types and default values of the metrics
def custom_metric_definitions():
    
    metrics = [MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[0],
                                    thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
              MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[1],
                                 thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.6),MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1)]),
              MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[2],
                                 thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.6),MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1)])]
    #Comment the below tags code if there are no tags to be created
    tags = [MonitorTagRequest(name=TAGS[0], description=TAG_DESCRIPTION[0])]
    
    return metrics, tags

In [ ]:
def create_custom_monitor_definition():
    # check if the custom monitor definition already exists or not
    existing_definition = get_custom_monitor_definition()

    # if it does not exists, then create a new one.
    if existing_definition is None:
        metrics, tags = custom_metric_definitions()
        custom_monitor_details = wos_client.monitor_definitions.add(name=CUSTOM_MONITOR_NAME, metrics=metrics, tags=tags, background_mode=False).result
    else:
        # otherwise, send the existing definition
        custom_monitor_details = existing_definition
    return custom_monitor_details

In [ ]:
custom_monitor_details = create_custom_monitor_definition()
custom_monitor_id = custom_monitor_details.metadata.id
custom_monitor_id

### Check the existence of custom monitor instance.


In [ ]:
def get_custom_monitor_instance(custom_monitor_id):
    monitor_instances = wos_client.monitor_instances.list(data_mart_id = WOS_GUID, monitor_definition_id = custom_monitor_id, target_target_id = subscription_id).result.monitor_instances
    if len(monitor_instances) == 1:
        return monitor_instances[0]
    return None

In [ ]:
# Openscale MRM service invokes custom metrics deployment url during runtime and wait for the default time of 60 second's to 
# to check the run status ie finished/Failed and fetch the latest measurement. Increase the wait time, if the runtime deployment 
# takes more than 60 seconds to compute and publish the custom metrics 

#Update the wait time here.
custom_metrics_wait_time = 120 #time in seconds <update the time here>
#specify the provider type to "wml_batch" for batch deployments
custom_metrics_provider_type = "wml_batch"

### Update the custom monitor instance.

In [ ]:
def update_custom_monitor_instance(custom_monitor_instance_id):
    payload = [
     {
       "op": "replace",
       "path": "/parameters",
       "value": {
           "custom_metrics_provider_id": integrated_system_id,
           "custom_metrics_provider_type": custom_metrics_provider_type,
           "custom_metrics_wait_time":   custom_metrics_wait_time,
           "space_id": space_id,
           "deployment_id": deployment_uid,
           "hardware_spec_id": hardware_spec_id
       }
     }
    ]
    response = wos_client.monitor_instances.update(custom_monitor_instance_id, payload, update_metadata_only = True)
    result = response.result
    return result

### For the custom monitor definition, create a custom monitor instance.


In [ ]:
def create_custom_monitor_instance(custom_monitor_id):
    # Check if an custom monitor instance already exists
    existing_monitor_instance = get_custom_monitor_instance(custom_monitor_id)

    # If it does not exist, then create one
    if existing_monitor_instance is None:
        target = Target(
                target_type=TargetTypes.SUBSCRIPTION,
                target_id=subscription_id
            )
        parameters = {
            "custom_metrics_provider_id": integrated_system_id,
            "custom_metrics_provider_type": custom_metrics_provider_type,
            "custom_metrics_wait_time":   custom_metrics_wait_time,
            "space_id": space_id,
            "deployment_id": deployment_uid,
            "hardware_spec_id": hardware_spec_id
        }
        # create the custom monitor instance id here.
        custom_monitor_instance_details = wos_client.monitor_instances.create(
                    data_mart_id=WOS_GUID,
                    background_mode=False,
                    monitor_definition_id=custom_monitor_id,
                    target=target,
                    parameters=parameters
        ).result
    else:
        # otherwise, update the existing one with latest integrated system details.
        instance_id = existing_monitor_instance.metadata.id
        custom_monitor_instance_details = update_custom_monitor_instance(instance_id)
    return custom_monitor_instance_details

In [ ]:
monitor_instance_details = create_custom_monitor_instance(custom_monitor_id)
custom_monitor_instance_id = monitor_instance_details.metadata.id
print(monitor_instance_details)

## Recap of the steps performed in this notebook

- Create a python function
- Deploy the python function to WML
- Create an OpenScale Integrated System pointing to the python function
- Create a Custom Monitor Definition mentioning various custom metrics
- Create a Custom Monitor Instance and specify the Integrated System ID in the monitor instance configuration.

### Upon publishing required payload logging or feedback logging data, please visit OpenScale console and perform `Evaluate Now` from Model Risk Management dashboard / Actions menu to evaluate the configured Custom Metrics Provider.

# [OPTIONAL EXECUTION STEP] Invoke the custom metrics deployment Python function.

Validate the custom metrics provider deployment by providing the correct set of paramaters to generate the custom metrics.

In [ ]:
import uuid
parameters = {
    "custom_metrics_provider_id": integrated_system_id,
    "custom_metrics_wait_time":   custom_metrics_wait_time,
    "custom_metrics_provider_type": custom_metrics_provider_type,
    "space_id": space_id,
    "deployment_id": deployment_uid,
    "hardware_spec_id": hardware_spec_id,
    "run_details": {
    "run_id": str(uuid.uuid4()),
    "run_status": "Running"
    }
}

payload= {
    "data_mart_id" : WOS_GUID,
    "subscription_id" : subscription_id,
    "custom_monitor_id" : custom_monitor_id,
    "custom_monitor_instance_id" : custom_monitor_instance_id,
    "custom_monitor_instance_params": parameters
    
}

input_data= { "input_data": [ {"values": [ payload ] } ]
            }
func_result = custom_metrics_provider()(input_data)
print(func_result)

## Congratulations

You have finished configuring Custom Monitor Definition and Monitor instance for your Subscription. You can now run the custom monitor from [Watson OpenScale Dashboard](http://aiopenscale.cloud.ibm.com). Click the tile of your model and select `Evaluate Now` option from `Actions` drop down menu to run the monitor.